In [ ]:
# import os

# This is needed due to zipkin issues related to protobuf.
# os.environ["PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION"] = "python"

import os
from pathlib import Path
import sys

import dotenv

# Add base dir to path to be able to access test folder.
base_dir = Path().cwd().parent.parent.resolve()
if str(base_dir) not in sys.path:
    print(f"Adding {base_dir} to sys.path")
    sys.path.append(str(base_dir))

os.environ["OTEL_PYTHON_LOGGING_AUTO_INSTRUMENTATION_ENABLED"] = "true"

dotenv.load_dotenv()

# Deps, OTEL
# ! pip install opentelemetry-api opentelemetry-sdk opentelemetry-exporter-otlp

# OTEL contrib instrumentors
#  opentelemetry-instrumentation-sqlalchemy opentelemetry-instrumentation-requests

# Traceloop instrumentors
# ! pip install opentelemetry-instrumentation-llamaindex opentelemetry-instrumentation-openai

# Arize openinference instrumentors
# ! pip install "openinference-instrumentation-llama-index>=2"

# Install zipkin python package:
# ! pip install opentelemetry-exporter-zipkin-proto-http

# Start the zipkin docker container:
# ! docker run --rm -d -p 9411:9411 --name zipkin openzipkin/zipkin

# Stop the zipkin docker container:
# ! docker stop $(docker ps -a -q --filter ancestor=openzipkin/zipkin)

In [ ]:
import os
import urllib.request

from llama_index.core import Settings
from llama_index.core import SimpleDirectoryReader
from llama_index.core import VectorStoreIndex
from llama_index.llms.openai import OpenAI
from openinference.instrumentation.llama_index import (
    LlamaIndexInstrumentor as oi_LlamaIndexInstrumentor,
)
from opentelemetry import trace
from opentelemetry.exporter.zipkin.json import ZipkinExporter
from opentelemetry.instrumentation.llamaindex import LlamaIndexInstrumentor
from opentelemetry.instrumentation.openai import OpenAIInstrumentor
from opentelemetry.instrumentation.requests import RequestsInstrumentor
from opentelemetry.instrumentation.sqlalchemy import SQLAlchemyInstrumentor
from trulens.apps.llamaindex import TruLlama
from trulens.core import Select
from trulens.core.session import TruSession
from trulens.experimental.otel_tracing.core.trace import TracerProvider

# from opentelemetry.sdk.trace import TracerProvider

provider = TracerProvider()
# processor = BatchSpanProcessor(ConsoleSpanExporter())
# provider.add_span_processor(processor)

# Sets the global default tracer provider
trace.set_tracer_provider(provider)

# Creates a tracer from the global tracer provider
tracer = trace.get_tracer("example-tracer")

In [ ]:
url = "https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/paul_graham/paul_graham_essay.txt"
file_path = "data/paul_graham_essay.txt"

if not os.path.exists("data"):
    os.makedirs("data")

if not os.path.exists(file_path):
    urllib.request.urlretrieve(url, file_path)

In [ ]:
tru = TruSession(
    _experimental_otel_exporter=ZipkinExporter(
        endpoint="http://localhost:9411/api/v2/spans"
    )
)


TruSession().experimental_enable_feature("otel_tracing")

In [ ]:
oi_LlamaIndexInstrumentor().instrument()

LlamaIndexInstrumentor().instrument()
SQLAlchemyInstrumentor().instrument()
RequestsInstrumentor().instrument()
OpenAIInstrumentor().instrument()

Settings.chunk_size = 128
Settings.chunk_overlap = 16
Settings.llm = OpenAI()

documents = SimpleDirectoryReader("data").load_data()
index = VectorStoreIndex.from_documents(documents)

query_engine = index.as_query_engine(similarity_top_k=3)

In [ ]:
tru_query_engine_recorder = TruLlama(
    query_engine,
    app_name="LlamaIndex_App",
    app_version="base",
)

In [ ]:
# or as context manager
with tru_query_engine_recorder as recording:
    with tracer.start_as_current_span("Querying LlamaIndex") as span:
        span.set_attribute("custom_attribute", "This can by anything.")

        query_engine.query("What did the author do growing up?")

In [ ]:
rec = recording.get()

In [ ]:
# Check the span produced by openinference:
rec.get(Select.RecordSpans.OpenAI.chat.attributes)

In [ ]:
# Check the span produced by traceloop:
rec.get(Select.RecordSpans.OpenAI.workflow.attributes)